# izmirwildfire2025 – Analiz Defteri

Tek defter akışı: GEE init, AOI, Sentinel‑2 ön işleme, NDVI/NBR, dNDVI/dNBR, şiddet ve folium haritalar.
Önkoşullar: `pip install -r requirements.txt` ve bir kez `earthengine authenticate`.


## 0. Önkoşullar

Bu defteri çalıştırmadan önce aşağıdaki kurulum adımlarını uygulayın.

- Python 3.10+ önerilir ve izole ortam kullanın (venv).
- Bağımlılıkları `requirements.txt` üzerinden kurun.

Örnek komutlar:

```bash
python -m venv .venv
. .venv/bin/activate  # Windows: .venv\\Scripts\\activate
pip install -r requirements.txt
```

Not: Bağımlılık çatışmalarını önlemek için her projede ayrı bir sanal ortam önerilir.


## 1. Earth Engine Kimlik Doğrulama (bir kez)

Google Earth Engine (GEE) API'yi kullanabilmek için önce kimlik doğrulaması yapın.

- Kullanıcı hesabı ile interaktif doğrulama:

```bash
earthengine authenticate
```

- Alternatif: Servis hesabı ile çalışacaksanız ortam değişkenlerini ayarlayın:

```bash
export EE_SERVICE_ACCOUNT="service-account@project.iam.gserviceaccount.com"
export EE_PRIVATE_KEY_FILE="/path/to/key.json"
export EE_PROJECT="your-project-id"
```

Ardından defterde `ee_init(project=...)` çağrısı yapılır (CLI ve pipeline buna göre çalışır).


In [ ]:
# Importlar
import os
from IPython.display import IFrame, display, Image, HTML
import pandas as pd

from gee.pipeline import run_pipeline
from gee.utils import ee_init
from gee.aoi import get_aoi
from gee.preprocess import prepare_composite
from gee.indices import with_indices
from gee.change import compute_diffs, classify_dnbr
from gee.visualize import export_report_pngs, export_truecolor_pngs


In [ ]:
# Earth Engine proje kimliği (opsiyonel) ve erken başlatma
ee_project = 'karabukwildfire2025'
try:
    ee_init(project=ee_project)
    print('Earth Engine initialized.')
except Exception as e:
    print('Earth Engine init error:', e)


In [ ]:
# Parametreler
# Tarihler ISO (YYYY-MM-DD)
pre_start, pre_end   = '2025-06-10', '2025-06-20'
post_start, post_end = '2025-07-25', '2025-08-15'
# AOI: kökteki GeoJSON; yoksa İzmir bbox kullanılır
aoi_path = 'gee/aoi.geojson'
# Çıktı klasörü ve alan çözünürlüğü (m)
out_dir = 'results'
area_scale = 10

print('AOI exists? ', os.path.exists(aoi_path))
print('OUT dir   : ', out_dir)
(pre_start, pre_end, post_start, post_end)


In [ ]:
# Doğrulama ve Hızlı Önizleme
from datetime import date
def _check_dates(a,b,c,d):
    da, db, dc, dd = map(date.fromisoformat, (a,b,c,d))
    assert da <= db, 'pre_start <= pre_end olmalı'
    assert dc <= dd, 'post_start <= post_end olmalı'
    return (da, db, dc, dd)
try:
    _check_dates(pre_start, pre_end, post_start, post_end)
    print('Tarih aralıkları geçerli.')
except Exception as e:
    print('Tarih hatası:', e)

if not os.path.exists(aoi_path):
    print('AOI dosyası bulunamadı, varsayılan İzmir bbox kullanılacak.')
aoi = get_aoi(aoi_path)
center = aoi.centroid(10).coordinates().getInfo()
print('AOI center (lon,lat):', center)
os.makedirs(out_dir, exist_ok=True)
print('Çıktı klasörü hazır:', out_dir)


In [ ]:
# Çalıştır
outputs = run_pipeline(
    pre_start=pre_start, pre_end=pre_end,
    post_start=post_start, post_end=post_end,
    aoi_geojson=aoi_path, out_dir=out_dir,
    project=ee_project,
    area_scale=area_scale
)
outputs


In [ ]:
# Haritaları defter içinde görüntüle (opsiyonel)
from IPython.display import HTML

def show_html(path, height=600):
    try:
        display(IFrame(src=path, width='100%', height=height))
    except Exception:
        try:
            with open(path, 'r', encoding='utf-8') as f:
                html = f.read()
            display(HTML(html))
        except Exception as e:
            display(HTML(f"<div style='color:#a00'>Unable to display {path}: {e}</div>"))


In [ ]:
# Özet istatistikler
pd.read_csv(outputs['summary_csv'])


In [ ]:
# Rapor PNG'leri (analiz katmanları + doğal renk)
from IPython.display import Image, HTML, display
import os, base64

aoi = get_aoi(aoi_path)
pre = with_indices(prepare_composite(aoi, pre_start, pre_end))
post = with_indices(prepare_composite(aoi, post_start, post_end))
diffs = compute_diffs(pre, post)
severity = classify_dnbr(diffs['dNBR'])

pngs = export_report_pngs(pre=pre, post=post, diffs=diffs, severity=severity, aoi=aoi, out_dir=out_dir)
rgbs = export_truecolor_pngs(pre=pre, post=post, aoi=aoi, out_dir=out_dir)

order = [
  ('Pre RGB', rgbs.get('pre_rgb_png')),
  ('Post RGB', rgbs.get('post_rgb_png')),
  ('Pre NDVI', pngs.get('pre_ndvi_png')),
  ('Post NDVI', pngs.get('post_ndvi_png')),
  ('dNBR', pngs.get('dnbr_png')),
  ('Severity', pngs.get('severity_png')),
]

def show_image(path):
    try:
        display(Image(filename=path))
    except Exception:
        try:
            with open(path,'rb') as f:
                data = base64.b64encode(f.read()).decode('ascii')
            display(HTML(f"<img src='data:image/png;base64,{data}' style='max-width:100%;height:auto;'/>"))
        except Exception as e:
            display(HTML(f"<div style='color:#a00'>Unable to display {path}: {e}</div>"))

for title, p in order:
    if not p or not os.path.exists(p):
        continue
    display(HTML(f"<h4 style='margin:8px 0'>{title}</h4>"))
    show_image(p)
